<a href="https://colab.research.google.com/github/EmilioRodrigues/CrewAI/blob/main/Crew_Lkdin_post_com_Docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U crewai crewai_tools openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client markdown


In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

# Obtendo as chaves das variáveis do Google Colab
openai_key = userdata.get("CREWAI")
serper_key = userdata.get("SERPER")

# Verificando se as chaves foram carregadas corretamente
if not openai_key:
    raise ValueError("❌ ERRO: A chave da OpenAI não foi carregada!")
if not serper_key:
    raise ValueError("❌ ERRO: A chave da SERPER não foi carregada!")

# Definindo as variáveis de ambiente
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

print("✅ Chaves de API configuradas com sucesso!")


✅ Chaves de API configuradas com sucesso!


In [ ]:
from google.colab import auth
import google.auth
from googleapiclient.discovery import build

# 🔐 Autenticar Google Drive
auth.authenticate_user()
creds, _ = google.auth.default()

# Criar clientes para Google Drive e Google Docs
drive_service = build("drive", "v3", credentials=creds)
docs_service = build("docs", "v1", credentials=creds)

print("✅ Autenticação concluída!")


✅ Autenticação concluída!


In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

AGENTES  - BUSCADOR / REDATOR / EDITOR


In [ ]:
buscador = Agent(
    role='Buscador de Conteúdo',
    goal='Buscar conteúdo online sobre {tema}',
    backstory=(
        "Estamos no ano de 2025 e você está trabalhando na criação de artigos para o LinkedIn sobre {tema}.\n"
        "Você vai fazer uma busca sobre informações na internet, coletá-las e agrupá-las.\n"
        "Seu trabalho servirá de base para o Redator de Conteúdo."
    ),
    model = 'gpt-3.5-turbo',
    tools=[search_tool, scrape_tool],
    verbose=True
)


In [ ]:
redator = Agent(
    role='Redator de Conteúdo',
    goal='Escreva um artigo divertido e factualmente correto sobre {tema}.',
    backstory=(
        "Você está trabalhando na redação de um artigo para o LinkedIn sobre {tema}.\n"
        "Você vai utilizar os dados coletados pelo Buscador de Conteúdo para escrever um artigo "
        "interessante, divertido, atualizado e factualmente correto para o LinkedIn.\n"
        "Você deve evitar fontes de pesquisa desatualizadas e fora de contexto com {tema}.\n"
        "Não invente nem crie nada que não possa ser confirmado como real e factível.\n"
        "Seu trabalho servirá de base para o Editor de Conteúdo."
    ),
    model = 'gpt-3.5-turbo',
    tools=[],  # O redator não precisa de ferramentas de busca/scraping
    verbose=True
)


In [ ]:
editor = Agent(
    role='Editor de Conteúdo',
    goal='Editar um texto para LinkedIn, tornando-o mais informal, sem perder o profissionalismo.',
    backstory=(
        "Você está trabalhando na edição de um artigo para o LinkedIn.\n"
        "Você vai receber um texto do Redator de Conteúdo e editá-lo para um tom de voz "
        "semelhante ao de Emilio Barbosa, que é mais informal e divertido.\n"
        "Mantenha um equilíbrio entre descontração e profissionalismo."
    ),
    model = 'gpt-3.5-turbo',
    tools=[],  # O editor não precisa de ferramentas de busca ou scraping
    verbose=True
)


TASKS

In [ ]:
buscar = Task(
    description=(
        "1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre {tema}.\n"
        "2. Identifique o público-alvo, considerando seus interesses e pontos de dor.\n"
        "3. Inclua palavras-chave de SEO e dados ou fontes relevantes e factuais.\n"
    ),
    agent=buscador,
    tools=[search_tool, scrape_tool],  # Usa as ferramentas do Buscador
    expected_output=(
        "Um relatório detalhado com:\n"
        "- As últimas tendências sobre {tema}.\n"
        "- O público-alvo e seus interesses.\n"
        "- Palavras-chave de SEO mais relevantes.\n"
        "- Fontes confiáveis e recentes utilizadas na pesquisa."
    )
)


In [ ]:
redigir = Task(
    description=(
        "1. Use os dados coletados pelo Buscador de Conteúdo para escrever um artigo sobre {tema}.\n"
        "   O artigo deve ser: interessante, divertido, atualizado e factualmente correto para o LinkedIn.\n"
        "2. Incorpore palavras-chave de SEO de forma natural ao longo do texto.\n"
        "3. Certifique-se de que o post esteja estruturado de forma cativante, com uma conclusão que faça o leitor refletir.\n"
        "4. Acrescente uma chamada à ação, para comentar e compartilhar o post"
    ),
    agent=redator,
    tools=[],  # O Redator não precisa de ferramentas
    expected_output=(
        "Um artigo completo e envolvente sobre {tema}, pronto para ser publicado no LinkedIn.\n"
        "O artigo deve conter:\n"
        "- Um título atraente\n"
        "- Uma introdução envolvente\n"
        "- Conteúdo bem estruturado\n"
        "- Uso natural de palavras-chave de SEO\n"
        "- Uma conclusão que estimule a reflexão do leitor."
    )
)


In [ ]:
editar = Task(
    description=(
        "Revisar a postagem do LinkedIn quanto a:\n"
        "- Erros gramaticais.\n"
        "- Alinhamento com a voz pessoal de Emilio Rodrigues.\n"
        "- Aplicar formatação Markdown, incluindo títulos, listas, negrito e itálico.\n"
        "- O texto final deve estar pronto para ser copiado e colado em um post do LinkedIn com formatação correta."
    ),
    agent=editor,
    tools=[],  # O Editor não precisa de ferramentas externas
    expected_output=(
        "Um post formatado corretamente em Markdown, incluindo:\n"
        "- **Título principal** com `#`\n"
        "- **Seções bem definidas** com `##`\n"
        "- **Palavras-chave em negrito** (`**palavra**`)\n"
        "- **Listas ordenadas e não ordenadas** (`-`, `1.`)\n"
        "- **Blocos de citação** (`> Texto citado`)\n"
    )
)


CRIANDO O CREW

In [ ]:
import logging
logging.getLogger("opentelemetry.trace").setLevel(logging.ERROR)


In [ ]:
from crewai import Crew, Process  # Importando Crew e Process

equipe = Crew(
    agents=[buscador, redator, editor],
    tasks=[buscar, redigir, editar],
    process=Process.sequential,  # Define execução sequencial
    verbose=True
)


CONTAGEM DE TOKENS

In [ ]:
import os
import openai
import logging
from IPython.display import display, Markdown

# Ativar logs detalhados
logging.basicConfig(level=logging.INFO)

# Criar um cliente OpenAI com log ativado
client = openai.OpenAI(api_key=os.getenv("CREWAI"))

# Função para contar tokens de um único modelo
def contar_tokens(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )

    tokens_entrada = response.usage.prompt_tokens
    tokens_saida = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens

    # Criar Markdown para visualização clara
    resultado_markdown = f"""
    ## 📊 Contagem de Tokens
    ### `{model}`
    - 📥 Entrada: `{tokens_entrada}`
    - 📤 Saída: `{tokens_saida}`
    - ⚡ Total: `{total_tokens}`
    """

    display(Markdown(resultado_markdown))  # Exibir no formato Markdown

# Teste rápido com um único modelo
messages = [
    {"role": "system", "content": "Você é um assistente que gera textos sobre alimentação saudável."},
    {"role": "user", "content": "Quais são as principais tendências da alimentação saudável?"}
]

contar_tokens(messages, model="gpt-3.5-turbo")  # Agora apenas com o gpt-3.5-turbo



    ## 📊 Contagem de Tokens
    ### `gpt-3.5-turbo`
    - 📥 Entrada: `43`
    - 📤 Saída: `349`
    - ⚡ Total: `392`
    

START CREW

In [ ]:
tema_do_artigo = "Etarismo na transição de carreira"
entradas = {"tema": tema_do_artigo}

# 🚀 Rodar o CrewAI para gerar o artigo final
resultado = equipe.kickoff(inputs=entradas)

# 📊 Contar os tokens usados (corrigido para apenas um modelo)
contar_tokens(messages, model="gpt-3.5-turbo")


# Agent: Buscador de Conteúdo
## Task: 1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre Etarismo na transição de carreira.
2. Identifique o público-alvo, considerando seus interesses e pontos de dor.
3. Inclua palavras-chave de SEO e dados ou fontes relevantes e factuais.



# Agent: Buscador de Conteúdo
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"latest trends ageism career transition 2025\"}"
## Tool Output: 
{'searchParameters': {'q': 'latest trends ageism career transition 2025', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Experts Weigh In: Why 2025 Holds Hope for Older Workers and ...', 'link': 'https://www.linkedin.com/pulse/experts-weigh-why-2025-holds-hope-older-workers-janine-vanderburg-tytkc', 'snippet': 'There is a strong recognition that we need policies that encourage age inclusion and that we need to strengthen laws against age discrimination.', 'position': 1},


    ## 📊 Contagem de Tokens
    ### `gpt-3.5-turbo`
    - 📥 Entrada: `43`
    - 📤 Saída: `368`
    - ⚡ Total: `411`
    

CRIANDO O ARQUIVO GOOGLE DOCS

In [ ]:
import markdown
from googleapiclient.discovery import build
from IPython.display import display, Markdown

def salvar_artigo_no_google_docs(titulo, conteudo_markdown):
    """
    Converte um texto Markdown e cria um Google Docs com o conteúdo formatado.
    """
    # 🚀 Criar um novo documento no Google Docs
    doc = docs_service.documents().create(body={"title": titulo}).execute()
    doc_id = doc["documentId"]

    # 📝 Adicionar conteúdo ao documento
    requests = [
        {
            "insertText": {
                "location": {"index": 1},
                "text": conteudo_markdown.replace("\n", "\n")  # Mantém formatação
            }
        }
    ]

    docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

    # 📎 Link para o documento criado
    link = f"https://docs.google.com/document/d/{doc_id}/edit"
    print(f"✅ Artigo salvo no Google Docs: {link}")
    return link


SALVANDO ARQUIVO DO GOOGLE DOCS

In [ ]:
# 📝 Salvar o artigo gerado no Google Docs
titulo = "Etarismo na transição de carreira"
conteudo_markdown = "\n\n".join(str(task) for task in resultado.tasks_output)  # Conversão corrigida

# Salvar no Google Docs e exibir link
link_doc = salvar_artigo_no_google_docs(titulo, conteudo_markdown)
display(Markdown(f"📄 **[Acesse o artigo aqui]({link_doc})**"))


✅ Artigo salvo no Google Docs: https://docs.google.com/document/d/1yq4OlAY-O2Hd1GSjaCAmzTSASJZwWjQNfH1_ewqETCM/edit


📄 **[Acesse o artigo aqui](https://docs.google.com/document/d/1yq4OlAY-O2Hd1GSjaCAmzTSASJZwWjQNfH1_ewqETCM/edit)**

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# 🔹 Interface interativa para criar artigos
def gerar_artigo_interativo(titulo):
    if not titulo.strip():
        display(Markdown("❌ **Erro: Insira um título válido para o artigo!**"))
        return

    entradas = {"tema": titulo}

    # 🚀 Rodar CrewAI para gerar o artigo
    resultado = equipe.kickoff(inputs=entradas)

    # 📊 Contar tokens usados
    messages = [
        {"role": "system", "content": "Você é um assistente que gera textos."},
        {"role": "user", "content": f"Crie um artigo sobre {titulo}"}
    ]
    contar_tokens(messages, model="gpt-3.5-turbo")  # Função já definida anteriormente

    # 📄 Salvar no Google Docs
    conteudo_markdown = "\n\n".join(str(task) for task in resultado.tasks_output)
    link_doc = salvar_artigo_no_google_docs(titulo, conteudo_markdown)  # Função já definida anteriormente

    # 📎 Exibir link do artigo
    display(Markdown(f"📄 **[Acesse o artigo pronto aqui]({link_doc})**"))

    # 🔄 Reexibir a interface para novo artigo
    display(titulo_input, botao_gerar)

# 🔹 Criar interface no Google Colab
titulo_input = widgets.Text(
    value="",
    placeholder="Digite o título do artigo...",
    description="📝 Título:",
    layout=widgets.Layout(width="50%")
)

botao_gerar = widgets.Button(description="🚀 Gerar Artigo", button_style="success")

def on_button_click(b):
    gerar_artigo_interativo(titulo_input.value)

botao_gerar.on_click(on_button_click)

# Exibir interface inicial
display(titulo_input, botao_gerar)


Text(value='', description='📝 Título:', layout=Layout(width='50%'), placeholder='Digite o título do artigo...'…

Button(button_style='success', description='🚀 Gerar Artigo', style=ButtonStyle())

# Agent: Buscador de Conteúdo
## Task: 1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre Cuidado com pets em 2025.
2. Identifique o público-alvo, considerando seus interesses e pontos de dor.
3. Inclua palavras-chave de SEO e dados ou fontes relevantes e factuais.



# Agent: Buscador de Conteúdo
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"pet care trends 2025\"}"
## Tool Output: 
{'searchParameters': {'q': 'pet care trends 2025', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Paws and Reflect: 2025 Pet Trends in a Post-Pandemic World', 'link': 'https://www.numerator.com/resources/blog/pet-industry-trends-2025/', 'snippet': 'With a 9.2% increase in buy rate, pet wellness is driven by everything from skin care to dental care. Both new and existing pet owners are ...', 'position': 1}, {'title': 'Trends For 2025 - Val Talks Pets', 'link': 'https://valtalkspets.com/2025/01/03/trends


    ## 📊 Contagem de Tokens
    ### `gpt-3.5-turbo`
    - 📥 Entrada: `37`
    - 📤 Saída: `737`
    - ⚡ Total: `774`
    

✅ Artigo salvo no Google Docs: https://docs.google.com/document/d/1Y_aLJFTEnhdykGFH57KjlYxwdey2XImEtvOsR2Zh6T0/edit


...


    ## 📊 Contagem de Tokens
    ### `gpt-3.5-turbo`
    - 📥 Entrada: `43`
    - 📤 Saída: `479`
    - ⚡ Total: `522`
    

✅ Artigo salvo no Google Docs: https://docs.google.com/document/d/1WrXvnKbl3T-24stRo47V8ih5eUhCf2TE0WF0odLRoKU/edit


📄 **[Acesse o artigo pronto aqui](https://docs.google.com/document/d/1WrXvnKbl3T-24stRo47V8ih5eUhCf2TE0WF0odLRoKU/edit)**

Text(value='A igreja evangelica brasilieira e seus escandalos', description='📝 Título:', layout=Layout(width='…

Button(button_style='success', description='🚀 Gerar Artigo', style=ButtonStyle())